# Práctica 4 - Isac Añor Santana

In [1]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import ModifiedFaceDetector
import random

## Tarea práctica 4 - Filtro de payaso
Solamente se usa un detector de caras, dlib, pro su facilidad para acceder a los múltiples puntos detectados de la cara.



In [13]:


normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = ModifiedFaceDetector.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

debug = 0

#Set camera resolution
cap.set(3,640)
cap.set(4,480)

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame)
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                try:
                    # Adding eyebrows
                    for i in range(17, 27):
                        (x,y) = shape[i]
                        cv2.ellipse(frame, (x-10, y-80), (60, 15), 90, 0, 360, (random.randint(0,255), random.randint(0,255), random.randint(0,255)), -1)

                    # Adding lipstick
                    for i in range(48, 60):
                        (start_x,start_y) = shape[i]
                        (end_x, end_y) = shape[i+1]
                        cv2.line(frame, (start_x,start_y), (end_x, end_y), (0,0,200), 15)
                    
                    # Adding nose
                    (x,y) = shape[30]
                    cv2.circle(frame, (x, y), 25, (50,50,255), -1)
                except Exception as e:
                    print(e)
                    pass
                
                # Adding glasses
                cv2.circle(frame, ((int)(lex), (int)(ley)), 30, (0, 0, 0), 2)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 30, (0, 0, 0), 2)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                #cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, "a", (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
